In [2]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.linear_model import LinearRegression

In [6]:
# Función para resetear columnas

def reset_columns(df):
    df.columns = ['_'.join(col) for col in df.columns.values]
    return df


# Función para ponderar a total población - individuos en millones las variables de condiciones de vida y otras
def weights(dfw,weight_varlist,factor):
    for var in weight_varlist:
        dfw[var] = dfw[var] * dfw[factor]
    return dfw


# Dataset para visualización

In [7]:
df = pd.read_csv('./Files/ECV_2004_2018.csv.gz', compression = 'gzip', sep = ';')
df2 = df.copy()

# Selección de las variables necesarias para visualización y para el modelo
df2 = df2[(df2['Year'] == 2013) | (df2['Year'] == 2018) |
      (df2['Year'] == 2008) | (df2['Year'] == 2009) |
      (df2['Year'] == 2010) | (df2['Year'] == 2011) |
      (df2['Year'] == 2012) | (df2['Year'] == 2013) |
      (df2['Year'] == 2014) | (df2['Year'] == 2015) |
      (df2['Year'] == 2016) | (df2['Year'] == 2017)][['Year','Year_IndID',
        'HHHolidays_No',
         'HHHolidays_Yes',
         'HHFood_No',
         'HHFood_Yes',
         'HHReserves_Yes',
         'HHReserves_No',                                           
         'HHComputer_No',
         'HHComputer_Yes',
         'HHWashMachine_No',
         'HHWashMachine_Yes',
         'HHCar_No',
         'HHCar_Yes',
         'HousingCost_HighImpactHH',
         'HousingCost_LowImpactHH',
         'HousingCost_MediumImpactHH',
         'HHHeath_No',
         'HHHeath_Yes',
         'vhPobreza_vhPobreza_No',
         'vhPobreza_vhPobreza_Yes',
         'vhMATDEP_vhMATDEP_No',
         'vhMATDEP_vhMATDEP_Yes',
         'WorStatus_EmployeeFT',
         'WorStatus_Student',
         'WorStatus_Unemployed',
         'FStudies_<Primaria',
         'FStudies_EdSuperior',
         'CrConditions_NChronic',
         'CrConditions_YChronic',
         'HLimitations_NoLimited',
         'HLimitations_NoSerLimitedG_limitado',
         'HLimitations_SerLimited',
         'MDClothes_No affordable',
         'MDClothes_Yes',
         'MDShoes_No affordable',
         'MDShoes_Yes',
         'MDFriends_No affordable',
         'MDFriends_Yes',
         'MDLeisure_No affordable',
         'MDLeisure_Yes',
         'MDSelf_No affordable',
         'MDSelf_Yes',
         'MDInternet_No affordable',
         'MDInternet_Yes',
         'PIB_Nacional',
         'Pob_Nacional',
         'PIB_percapita_Nacional',
         'PIB(%)_change_Nacional',
         'HHHolidays',
         'HHFood',
         'HHReserves',
         'HHCar',
         'HHHeath',
         'vhPobreza',
         'vhMATDEP',
         'MDClothes',
         'MDShoes',
         'MDFriends',
         'MDLeisure',
         'MDSelf', 
         'MDInternet',
         'CHealth', 
         'AREMonth', 
         'vhRentaa']].dropna()

CPU times: user 5.44 s, sys: 1.95 s, total: 7.39 s
Wall time: 7.37 s


# Estimación de la felicidad

In [11]:
df_model = pd.read_csv('./Files/ECV_2004&2018.csv.gz', compression = 'gzip', sep = ';')
df_model.head()

,Year,HHId,HHHolidays,HHFood,HHReserves,HHPhone,HHTV,HHComputer,HHWashMachine,HHCar,AREMonth,HousingCost,HHHeath,HHnMembers,HHConsUnits,vhRentaa,vhRentaAIa,vhPobreza,vhMATDEP,Year_HHid,Media_renta,Std_renta,vhRentaa_zscore,vhRentaa_xperson,Media_renta_ucons,Std_renta_ucons,vhRentaa_uncons_zscore,Media_renta2,Std_renta2,vhRentaa2_zscore,vhRentaa_xperson2,Media_renta_ucons2,Std_renta_ucons2,vhRentaa_uncons2_zscore,AREMonth_D,Region,Habitat,Comunidad Autónoma_x,year_x,PIB_,Pob_,PIB_percapita_,PIB(%)_change_,PIB(%)_varnacional_,Comunidad Autónoma_y,year_y,PIB_Nacional,Pob_Nacional,PIB_percapita_Nacional,PIB(%)_change_Nacional,IndID,Sex,YBirht,MarStatus,WorStatus,FStudies,CHealth,CrConditions,HLimitations,MDClothes,MDShoes,MDFriends,MDLeisure,MDSelf,MDInternet,FFTense,FFDown,FFCalm,FFDepressed,FFHappy,WSOovsat,WBSrelations,WBSowntime,WSBeconomy,Year_IndID,AgeNumeric,CHealth_D,HHHolidays_No,HHHolidays_Unknown / Not Declared,HHHolidays_Yes,HHFood_No,HHFood_Unknown / Not Declared,HHFood_Yes,HHReserves_No,HHReserves_Unknown / Not Declared,HHReserves_Yes,HHPhone_No,HHPhone_No_otros,HHPhone_Unknown / Not Declared,HHPhone_Yes,HHTV_No,HHTV_No_otros,HHTV_Unknown / Not Declared,HHTV_Yes,HHComputer_No,HHComputer_No_otros,HHComputer_Unknown / Not Declared,HHComputer_Yes,HHWashMachine_No,HHWashMachine_No_otros,HHWashMachine_Unknown / Not Declared,HHWashMachine_Yes,HHCar_No,HHCar_No_otros,HHCar_Unknown / Not Declared,HHCar_Yes,HousingCost_HighImpactHH,HousingCost_LowImpactHH,HousingCost_MediumImpactHH,HousingCost_Unknown / Not Declared,HHHeath_No,HHHeath_Unknown / Not Declared,HHHeath_Yes,vhPobreza_Unknown / Not Declared,vhPobreza_vhPobreza_No,vhPobreza_vhPobreza_Yes,vhMATDEP_vhMATDEP_No,vhMATDEP_vhMATDEP_Yes,Sex_Hombre,Sex_Mujer,MarStatus_Casado,MarStatus_Divorciado,MarStatus_Separado,MarStatus_Soltero,MarStatus_Uknown / Prefer not to say,MarStatus_Viudo,WorStatus_EmployeeFT,WorStatus_EmployeePT,WorStatus_HHoldKeeper,WorStatus_OtherNonWorking,WorStatus_Retired,WorStatus_SEmployedFT,WorStatus_SEmployedPT,WorStatus_Student,WorStatus_Uknown / Not declared,WorStatus_Unemployed,FStudies_<Primaria,FStudies_EdSuperior,FStudies_OrGeneral,FStudies_OrProfesional,FStudies_Primaria,FStudies_Secundaria1,FStudies_Secundaria2,FStudies_SecundariaNoOb,FStudies_Uknown,CrConditions_NChronic,CrConditions_Unknown / Not Declared,CrConditions_YChronic,HLimitations_NoLimited,HLimitations_NoSerLimitedG_limitado,HLimitations_SerLimited,HLimitations_Unknown / Not declared,MDClothes_No affordable,"MDClothes_No, other reason",MDClothes_Unknown / Not Declared,MDClothes_Yes,MDShoes_No affordable,"MDShoes_No, other reason",MDShoes_Unknown / Not Declared,MDShoes_Yes,MDFriends_No affordable,"MDFriends_No, other reason",MDFriends_Unknown / Not Declared,MDFriends_Yes,MDLeisure_No affordable,"MDLeisure_No, other reason",MDLeisure_Unknown / Not Declared,MDLeisure_Yes,MDSelf_No affordable,"MDSelf_No, other reason",MDSelf_Unknown / Not Declared,MDSelf_Yes,MDInternet_No affordable,"MDInternet_No, other reason",MDInternet_Unknown / Not Declared,MDInternet_Yes,CHealth_D_H_Bad,CHealth_D_H_Good,CHealth_D_H_NorGood_Bad,CHealth_D_H_VeryBad,CHealth_D_H_VeryGood,CHealth_D_Unknown / Not Declared,AREMonth_D_EM_Dificult,AREMonth_D_EM_Easy,AREMonth_D_EM_SW_Dificult,AREMonth_D_EM_SW_Easy,AREMonth_D_EM_VeryEasy,AREMonth_D_EM_Verydificult,AREMonth_D_Unknown / Not Declared,LifeSatisfaction0,LifeSatisfaction1,LifeSatisfaction2,LifeSatisfaction_int
0,2013,1,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,5.0,HighImpactHH,Yes,4,2.3,88450.00,92796.64,vhPobreza_No,vhMATDEP_No,2013_1,27812.14777,20714.19704,2.927357,38456.521739,16087.279759,11124.942229,2.010729,31722.605629,21228.622452,2.876966,40346.365217,18571.3444,11490.840175,1.894989,Fácil,País Vasco,Small,País Vasco,2013,61625767,2191682,28.118024,-1.428074,30.024202,España,2013,1019192082,47129783,21.625223,-0.766369,101,Mujer,1964,Casado,EmployeeFT,Uknown,1.0,NChronic,NoLimited,Yes,Yes,Yes,Yes,Yes,Yes,4.0,5.0,2.0,5.0,2.0,8.0,8.0,8.0,8.0,2013_101,49,Muy buena,0

In [12]:
# df para el modelo
df_model = pd.read_csv('./Files/ECV_2004&2018.csv.gz', compression = 'gzip', sep = ';')

# Variables modelo

X2 = df_model[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 'HHCar_Yes', 'HousingCost_HighImpactHH',
              'HousingCost_MediumImpactHH', 'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes','vhRentaa',
              'CrConditions_NChronic','HLimitations_NoLimited','CHealth','AREMonth']]
y = df_model['LifeSatisfaction2']


# Predicciones
reg2 = LinearRegression()

reg2.fit(X2, y)

df2['Predictions'] =reg2.predict(df2[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 'HHCar_Yes', 
                                              'HousingCost_HighImpactHH','HousingCost_MediumImpactHH', 'HHHeath_Yes', 
                                              'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes','vhRentaa','CrConditions_NChronic',
                                              'HLimitations_NoLimited','CHealth','AREMonth']])

# Visualización de la felicidad media por quitiles

In [18]:
# Cambio nombre variable
df2.rename(columns = {'PIB(%)_change_Nacional': 'PIB_change_Nacional'}, inplace = True) 
# Ponderación año
df_weigths = (df2.groupby(['Year'])['Pob_Nacional'].mean() \
              / df2.groupby(['Year'])['Year_IndID'].count()).reset_index().rename(columns = {0:'Weight'})
df2 = df2.merge(df_weigths)


# Creación de quintiles con las predicciones
df2['Quintiles_bins'] = pd.qcut(df2['Predictions'],5) # Distribuir todas las observaciones predecidas en 5 quintiles

# Asignación de quintil según los rangos resultantes
df2['Quintiles'] = df2['Quintiles_bins'].apply(lambda x: '1º - Inferior' if str(x) == '(3.694, 6.298]' \
                                                        else '2º' if str(x) ==  '(6.298, 6.874]' \
                                                        else '3º' if str(x) ==  '(6.874, 7.348]' \
                                                        else '4º' if str(x) ==  '(7.348, 7.725]' \
                                                        else '5º - Superior')

# dfs para data set de visualización centrado en el contexto, datos globales de economía y 

df_context_sum = df2.groupby(['Year','Quintiles'])['Weight','PIB_percapita_Nacional',
                            'PIB_change_Nacional'].sum().reset_index()
df_context_mean = df2.groupby(['Year'])[['Predictions']].mean().reset_index()

df_context =df_context_sum.merge(df_context_mean)

# Matriz de correlaciones para visualización

In [22]:
# Variables en la matriz de correlaciones
var_corr = ['HHHolidays_No',
 'HHHolidays_Yes',
 'HHFood_No',
 'HHFood_Yes',
 'HHReserves_Yes',
 'HHReserves_No',
 'HHComputer_No',
 'HHComputer_Yes',
 'HHWashMachine_No',
 'HHWashMachine_Yes',
 'HHCar_No',
 'HHCar_Yes',
 'HousingCost_HighImpactHH',
 'HousingCost_LowImpactHH',
 'HousingCost_MediumImpactHH',
 'HHHeath_No',
 'HHHeath_Yes',
 'vhPobreza_vhPobreza_No',
 'vhPobreza_vhPobreza_Yes',
 'vhMATDEP_vhMATDEP_No',
 'vhMATDEP_vhMATDEP_Yes',
 'WorStatus_EmployeeFT',
 'WorStatus_Student',
 'WorStatus_Unemployed',
 'FStudies_<Primaria',
 'FStudies_EdSuperior',
 'CrConditions_NChronic',
 'CrConditions_YChronic',
 'HLimitations_NoLimited',
 'HLimitations_NoSerLimitedG_limitado',
 'HLimitations_SerLimited',
 'MDClothes_No affordable',
 'MDClothes_Yes',
 'MDShoes_No affordable',
 'MDShoes_Yes',
 'MDFriends_No affordable',
 'MDFriends_Yes',
 'MDLeisure_No affordable',
 'MDLeisure_Yes',
 'MDSelf_No affordable',
 'MDSelf_Yes',
 'MDInternet_No affordable',
 'MDInternet_Yes','LifeSatisfaction2']

# Diccionario de nombres de la matriz de correlaciones 
dict_corr ={'LifeSatisfaction2':'Felicidad',
                         'HHHolidays_Yes':'Vacaciones_Sí',
                         'HHFood_Yes':'Alimentación Sí',
                         'HHReserves_Yes':'"Colchón" económico Sí',
                         'HHReserves_No':'"Colchón" económico No',
                         'HHCar_Yes':'Coche Sí',
                         'HHHeath_Yes':'Calefacción Sí',
                         'vhPobreza_vhPobreza_No':'Riesgo de pobreza No',
                         'vhMATDEP_vhMATDEP_No':'Carencia Material Severa No',
                         'MDClothes_Yes': 'Compra de ropa Sí',
                         'MDShoes_Yes':'Compra de zapatos Sí',
                         'MDFriends_Yes': 'Ocio con amigos Sí',
                         'MDLeisure_Yes': 'Ocio en general Sí',
                         'MDSelf_Yes':'Gasto en uno mismo Sí',
                         'MDInternet_Yes': 'Acceso a internet Sí',
                         'HHHolidays_No':'Vacaciones No', 
                         'HHFood_No':'Alimentación No', 
                         'HHComputer_No':'Ordenador No', 
                         'HHComputer_Yes':'Ordenador Sí',
                         'HHWashMachine_No':'Lavadora Sí',
                         'HHWashMachine_Yes':'Lavadora No', 
                         'HHCar_No': 'Coche No',
                         'HousingCost_HighImpactHH':'Alto impacto coste vivienda', 
                         'HousingCost_LowImpactHH':'Bajo impacto coste vivienda',
                         'HousingCost_MediumImpactHH':'Impacto medio coste vivienda',
                         'HHHeath_No':'Calefacción No', 
                         'vhPobreza_vhPobreza_Yes': 'Riesgo de pobreza Sí', 
                         'vhMATDEP_vhMATDEP_Yes': 'Carencia material severa',
                         'WorStatus_EmployeeFT': 'Trabajador a tiempo completo', 
                         'WorStatus_Student': 'Estudiante',
                         'WorStatus_Unemployed': 'Parado', 
                         'FStudies_<Primaria': 'Estudios < Primaria',
                         'FStudies_EdSuperior': 'Estudios Educación Superior',
                         'CrConditions_NChronic': 'Enfermedades Crónicas No',
                         'CrConditions_YChronic':'Enfermedades Crónicas Sí', 
                         'HLimitations_NoLimited': 'Limitaciones físicas No',
                         'HLimitations_NoSerLimitedG_limitado' : 'Limitaciones físicas leves Sí', 
                         'HLimitations_SerLimited': 'Limitaciones físicas graves Sí',
                         'MDClothes_No affordable' : 'Compra de ropa No',
                         'MDShoes_No affordable' : 'Compra de zapatos No',
                         'MDFriends_No affordable': 'Ocio con amigos No',
                         'MDLeisure_No affordable': 'Ocio en general No',
                         'MDSelf_No affordable': 'Gasto en uno mismo No',
                         'MDInternet_No affordable': 'Acceso a internet No'}


# Etiqueta codificación categoría

# Etiqueta codificación positivo / negativo

df_corr = df_model[var_corr].corr().stack().reset_index() \
              .rename(columns = {0:'correlation','level_0':'variable_1','level_1':'variable_2'})
df_corr['correlation_label'] = df_corr['correlation'].map('{:.2f}'.format)
df_corr['variable_1'] = df_corr['variable_1'].map(dict_corr)
df_corr['variable_2'] = df_corr['variable_2'].map(dict_corr)


df_corr.to_csv('./Files/correlations.csv', sep = ';',index = False)

# Dataset visualización condiciones de vida

In [26]:
df_weights = df2.copy()
df_means = df2.copy()
df_counts = df2.copy()

var_list2 = var_corr[:-1]


df_weights = weights(df_weights ,var_list2,'Weight')
df_means = weights(df_means,var_list2,'Predictions')


var_list3 = ['HHHolidays_No',
 'HHHolidays_Yes',
 'HHFood_No',
 'HHFood_Yes',
 'HHReserves_Yes',
 'HHReserves_No',
 'HHComputer_No',
 'HHComputer_Yes',
 'HHWashMachine_No',
 'HHWashMachine_Yes',
 'HHCar_No',
 'HHCar_Yes',
 'HousingCost_HighImpactHH',
 'HousingCost_LowImpactHH',
 'HousingCost_MediumImpactHH',
 'HHHeath_No',
 'HHHeath_Yes',
 'vhPobreza_vhPobreza_No',
 'vhPobreza_vhPobreza_Yes',
 'vhMATDEP_vhMATDEP_No',
 'vhMATDEP_vhMATDEP_Yes',
 'WorStatus_EmployeeFT',
 'WorStatus_Student',
 'WorStatus_Unemployed',
 'FStudies_<Primaria',
 'FStudies_EdSuperior',
 'CrConditions_NChronic',
 'CrConditions_YChronic',
 'HLimitations_NoLimited',
 'HLimitations_NoSerLimitedG_limitado',
 'HLimitations_SerLimited',
 'MDClothes_No affordable',
 'MDClothes_Yes',
 'MDShoes_No affordable',
 'MDShoes_Yes',
 'MDFriends_No affordable',
 'MDFriends_Yes',
 'MDLeisure_No affordable',
 'MDLeisure_Yes',
 'MDSelf_No affordable',
 'MDSelf_Yes',
 'MDInternet_No affordable',
 'MDInternet_Yes']

dictionary_map = {'LifeSatisfaction2':'Felicidad',
                         'HHHolidays_Yes':'Vacaciones_Sí',
                         'HHFood_Yes':'Alimentación Sí',
                         'HHReserves_Yes':'"Colchón" económico Sí',
                         'HHReserves_No':'"Colchón" económico No',
                         'HHCar_Yes':'Coche Sí',
                         'HHHeath_Yes':'Calefacción Sí',
                         'vhPobreza_vhPobreza_No':'Riesgo de pobreza No',
                         'vhMATDEP_vhMATDEP_No':'Carencia Material Severa No',
                         'MDClothes_Yes': 'Compra de ropa Sí',
                         'MDShoes_Yes':'Compra de zapatos Sí',
                         'MDFriends_Yes': 'Ocio con amigos Sí',
                         'MDLeisure_Yes': 'Ocio en general Sí',
                         'MDSelf_Yes':'Gasto en uno mismo Sí',
                         'MDInternet_Yes': 'Acceso a internet Sí',
                         'HHHolidays_No':'Vacaciones No', 
                         'HHFood_No':'Alimentación No', 
                         'HHComputer_No':'Ordenador No', 
                         'HHComputer_Yes':'Ordenador Sí',
                         'HHWashMachine_No':'Lavadora Sí',
                         'HHWashMachine_Yes':'Lavadora No', 
                         'HHCar_No': 'Coche No',
                         'HousingCost_HighImpactHH':'Alto impacto coste vivienda', 
                         'HousingCost_LowImpactHH':'Bajo impacto coste vivienda',
                         'HousingCost_MediumImpactHH':'Impacto medio coste vivienda',
                         'HHHeath_No':'Calefacción No', 
                         'vhPobreza_vhPobreza_Yes': 'Riesgo de pobreza Sí', 
                         'vhMATDEP_vhMATDEP_Yes': 'Carencia material severa',
                         'WorStatus_EmployeeFT': 'Trabajador a tiempo completo', 
                         'WorStatus_Student': 'Estudiante',
                         'WorStatus_Unemployed': 'Parado', 
                         'FStudies_<Primaria': 'Estudios < Primaria',
                         'FStudies_EdSuperior': 'Estudios Educación Superior',
                         'CrConditions_NChronic': 'Enfermedades Crónicas No',
                         'CrConditions_YChronic':'Enfermedades Crónicas Sí', 
                         'HLimitations_NoLimited': 'Limitaciones físicas No',
                         'HLimitations_NoSerLimitedG_limitado' : 'Limitaciones físicas leves Sí', 
                         'HLimitations_SerLimited': 'Limitaciones físicas graves Sí',
                         'MDClothes_No affordable' : 'Compra de ropa No',
                         'MDShoes_No affordable' : 'Compra de zapatos No',
                         'MDFriends_No affordable': 'Ocio con amigos No',
                         'MDLeisure_No affordable': 'Ocio en general No',
                         'MDSelf_No affordable': 'Gasto en uno mismo No',
                         'MDInternet_No affordable': 'Acceso a internet No'}

# Suma de condiciones sociales por el peso que tienen en la población real

df_grouped_sum = df_weights.groupby('Year')[var_list3].sum().reset_index()
df_grouped2_sum_m = pd.melt(df_grouped_sum, id_vars=['Year'], value_vars = var_list3)
df_grouped2_sum_m['variable'] = df_grouped2_sum_m['variable'].map(dictionary_map)

# Media de la satisfacción vital / Felicidad para cada una de esas condiciones
df_grouped_mean = df_means.groupby('Year')[var_list3].sum().reset_index()
df_grouped_mean['Year'] = df_grouped_mean['Year']**2
df_grouped_counts = df_counts.groupby('Year')[var_list3].sum().reset_index()
df_grouped_mean_c = df_grouped_mean / df_grouped_counts
df_grouped2_mean_m = pd.melt(df_grouped_mean_c, id_vars=['Year'], value_vars = var_list3)                      
df_grouped2_mean_m['variable'] = df_grouped2_mean_m['variable'].map(dictionary_map)

# df con ambos valores
df_conditions = df_grouped2_sum_m.merge(df_grouped2_mean_m, right_on = ['Year','variable'], 
                                        left_on = ['Year','variable'])

df_conditions.rename(columns = {'value_x':'Individuos', 'value_y': 'media_felicidad'}, inplace = True)

# merge con df de correlaciones

df_corr2 = df_corr.copy()
df_corr2 = df_corr2[df_corr2['variable_1'] == 'LifeSatisfaction2'].iloc[:-1] #df con las correlaciones de cada variable con la felicidad


df_conditions_corr = df_conditions.merge(df_corr2, left_on = 'variable', right_on = 'variable_2') \
            [['Year','variable','Individuos','correlation_label','media_felicidad']]

# merge con df de contexto
df_visualization = df_conditions_corr.merge(df_context)



# Codificaciones de variables
# Nombre de la variable
dictionary_variables = {'Vacaciones No': 'Vacaciones', 
                        'Vacaciones_Sí': 'Vacaciones',
                        'Alimentación No': 'Alimentación',
                        'Alimentación Sí': 'Alimentación',
                        '"Colchón" económico Sí': '"Colchón" económico',
                        '"Colchón" económico No': '"Colchón" económico',
                        'Ordenador No': 'Ordenador',
                        'Ordenador Sí': 'Ordenador',
                        'Lavadora Sí': 'Lavadora',
                        'Lavadora No': 'Lavadora', 
                        'Coche No' : 'Coche',
                        'Coche Sí' : 'Coche',
                        'Alto impacto coste vivienda': 'Vivienda',
                        'Bajo impacto coste vivienda': 'Vivienda',
                        'Impacto medio coste vivienda': 'Vivienda',
                        'Calefacción No': 'Calefacción', 
                        'Calefacción Sí': 'Calefacción', 
                        'Riesgo de pobreza No': 'Riesgo de pobreza',
                        'Riesgo de pobreza Sí': 'Riesgo de pobreza',
                        'Carencia Material Severa No': 'Carencia material',
                        'Carencia material severa': 'Carencia material',
                        'Trabajador a tiempo completo': 'Situación Laboral',
                        'Estudiante': 'Situación Laboral',
                        'Parado': 'Situación Laboral',
                        'Estudios < Primaria': 'Estudios',
                        'Estudios Educación Superior': 'Estudios',
                        'Enfermedades Crónicas No': 'Enfermedades Crónicas',
                        'Enfermedades Crónicas Sí': 'Enfermedades Crónicas',
                        'Limitaciones físicas No': 'Limitaciones Físicas',
                        'Limitaciones físicas leves Sí': 'Limitaciones Físicas',
                        'Limitaciones físicas graves Sí': 'Limitaciones Físicas',
                        'Compra de ropa No': 'Compra de ropa',
                        'Compra de ropa Sí': 'Compra de ropa',
                        'Compra de zapatos No': 'Compra de zapatos',
                        'Compra de zapatos Sí': 'Compra de zapatos',
                        'Ocio con amigos No': 'Ocio con amigos',
                        'Ocio con amigos Sí': 'Ocio con amigos',
                        'Ocio en general No': 'Ocio General',
                        'Ocio en general Sí': 'Ocio General',
                        'Gasto en uno mismo No': 'Gasto en uno mismo', 
                        'Gasto en uno mismo Sí': 'Gasto en uno mismo',
                        'Acceso a internet No': 'Internet', 
                        'Acceso a internet Sí': 'Internet'}

# Categorías
dictionary_category=  {'Vacaciones No': 'Economía no básica', 
                        'Vacaciones_Sí': 'Economía no básica',
                        'Alimentación No': 'Economía básica',
                        'Alimentación Sí': 'Economía básica',
                        '"Colchón" económico Sí': 'Situación económica',
                        '"Colchón" económico No': 'Situación económica',
                        'Ordenador No': 'Economía no básica',
                        'Ordenador Sí': 'Economía no básica',
                        'Lavadora Sí': 'Economía básica',
                        'Lavadora No': 'Economía básica', 
                        'Coche No' : 'Economía no básica',
                        'Coche Sí' : 'Economía no básica',
                        'Alto impacto coste vivienda': 'Situación económica',
                        'Bajo impacto coste vivienda': 'Situación económica',
                        'Impacto medio coste vivienda': 'Situación económica',
                        'Calefacción No': 'Economía básica', 
                        'Calefacción Sí': 'Economía básica', 
                        'Riesgo de pobreza No': 'Situación económica',
                        'Riesgo de pobreza Sí': 'Situación económica',
                        'Carencia Material Severa No': 'Situación económica',
                        'Carencia material severa': 'Situación económica',
                        'Trabajador a tiempo completo': 'Situación Laboral',
                        'Estudiante': 'Situación Laboral',
                        'Parado': 'Situación Laboral',
                        'Estudios < Primaria': 'Estudios',
                        'Estudios Educación Superior': 'Estudios',
                        'Enfermedades Crónicas No': 'Salud',
                        'Enfermedades Crónicas Sí': 'Salud',
                        'Limitaciones físicas No': 'Salud',
                        'Limitaciones físicas leves Sí': 'Salud',
                        'Limitaciones físicas graves Sí': 'Salud',
                        'Compra de ropa No': 'Economía básica',
                        'Compra de ropa Sí': 'Economía básica',
                        'Compra de zapatos No':'Economía básica',
                        'Compra de zapatos Sí': 'Economía básica',
                        'Ocio con amigos No': 'Economía no básica',
                        'Ocio con amigos Sí': 'Economía no básica',
                        'Ocio en general No': 'Economía no básica',
                        'Ocio en general Sí': 'Economía no básica',
                        'Gasto en uno mismo No': 'Economía no básica', 
                        'Gasto en uno mismo Sí': 'Economía no básica',
                        'Acceso a internet No': 'Economía no básica', 
                        'Acceso a internet Sí': 'Economía no básica'}

# Respuesta
dictionary_response =  {'Vacaciones No': 'No', 
                        'Vacaciones_Sí': 'Sí',
                        'Alimentación No': 'No',
                        'Alimentación Sí': 'Sí',
                        '"Colchón" económico Sí': 'Sí',
                        '"Colchón" económico No': 'No',
                        'Ordenador No': 'No',
                        'Ordenador Sí': 'Sí',
                        'Lavadora Sí': 'Sí',
                        'Lavadora No': 'No', 
                        'Coche No' : 'No',
                        'Coche Sí' : 'Sí',
                        'Alto impacto coste vivienda': 'Impacto Alto',
                        'Bajo impacto coste vivienda': 'Impacto Bajo',
                        'Impacto medio coste vivienda': 'Impacto Medio',
                        'Calefacción No': 'No', 
                        'Calefacción Sí': 'Sí', 
                        'Riesgo de pobreza No': 'No',
                        'Riesgo de pobreza Sí': 'Sí',
                        'Carencia Material Severa No': 'No',
                        'Carencia material severa': 'Sí',
                        'Trabajador a tiempo completo': 'tiempo completo',
                        'Estudiante': 'Estudiante',
                        'Parado': 'Situación Laboral',
                        'Estudios < Primaria': '< Primaria',
                        'Estudios Educación Superior': 'Educación Superior',
                        'Enfermedades Crónicas No': 'No',
                        'Enfermedades Crónicas Sí': 'Sí',
                        'Limitaciones físicas No': 'No',
                        'Limitaciones físicas leves Sí':  'Sí leves',
                        'Limitaciones físicas graves Sí':  'Sí graves',
                        'Compra de ropa No': 'No',
                        'Compra de ropa Sí':  'Sí',
                        'Compra de zapatos No':'No',
                        'Compra de zapatos Sí':  'Sí',
                        'Ocio con amigos No': 'No',
                        'Ocio con amigos Sí':  'Sí',
                        'Ocio en general No': 'No',
                        'Ocio en general Sí':  'Sí',
                        'Gasto en uno mismo No': 'No', 
                        'Gasto en uno mismo Sí':  'Sí',
                        'Acceso a internet No': 'No', 
                        'Acceso a internet Sí': 'Sí'}

# Incidencia positiva - negativa (comprobado con el coeficiente de correlación de cada una de las variables con la felicidad
dictionaryp_posneg  =  {'Vacaciones No': 'Negativo', 
                        'Vacaciones_Sí': 'Positivo',
                        'Alimentación No': 'Negativo',
                        'Alimentación Sí': 'Positivo',
                        '"Colchón" económico Sí': 'Positivo',
                        '"Colchón" económico No': 'Negativo',
                        'Ordenador No': 'Negativo',
                        'Ordenador Sí': 'Positivo',
                        'Lavadora Sí': 'Positivo',
                        'Lavadora No': 'Negativo', 
                        'Coche No' : 'Negativo',
                        'Coche Sí' : 'Positivo',
                        'Alto impacto coste vivienda': 'Negativo',
                        'Bajo impacto coste vivienda': 'Positivo',
                        'Impacto medio coste vivienda': 'Positivo',
                        'Calefacción No': 'Negativo', 
                        'Calefacción Sí': 'Positivo', 
                        'Riesgo de pobreza No': 'Positivo',
                        'Riesgo de pobreza Sí': 'Negativo',
                        'Carencia Material Severa No': 'Positivo',
                        'Carencia material severa': 'Negativo',
                        'Trabajador a tiempo completo': 'Positivo',
                        'Estudiante': 'Estudiante',
                        'Parado': 'Negativo',
                        'Estudios < Primaria': 'Negativo',
                        'Estudios Educación Superior': 'Positivo',
                        'Enfermedades Crónicas No': 'Positivo',
                        'Enfermedades Crónicas Sí': 'Negativo',
                        'Limitaciones físicas No': 'Positivo',
                        'Limitaciones físicas leves Sí': 'Negativo',
                        'Limitaciones físicas graves Sí':  'Negativo',
                        'Compra de ropa No': 'Negativo',
                        'Compra de ropa Sí': 'Positivo',
                        'Compra de zapatos No':'Negativo',
                        'Compra de zapatos Sí':  'Positivo',
                        'Ocio con amigos No': 'Negativo',
                        'Ocio con amigos Sí':  'Positivo',
                        'Ocio en general No': 'Negativo',
                        'Ocio en general Sí':  'Positivo',
                        'Gasto en uno mismo No': 'Negativo', 
                        'Gasto en uno mismo Sí':  'Positivo',
                        'Acceso a internet No': 'Negativo', 
                        'Acceso a internet Sí': 'Positivo'}

# Codificación de variables
df_visualization['variable_name'] = df_visualization['variable'].map(dictionary_variables)
df_visualization['variable_response'] = df_visualization['variable'].map(dictionary_response)
df_visualization['variable_category'] = df_visualization['variable'].map(dictionary_category)
df_visualization['variable_posneg'] = df_visualization['variable'].map(dictionary_category)

# Aplicación de los pesos de ponderación al df del modelo

In [27]:
df_model = df_model.merge(df_weigths) # Aplicación de los pesos de ponderación de cara a usarlo en alguna visualización
df_model.to_csv('./Files/Modelo.csv', sep = ';', index = False)